In [1]:
import numpy as np
import os
import math

import skimage
from skimage.io import imread, imsave
from skimage.transform import resize

from collections import Counter

In [7]:
image_1 = imread("D:/svruchtel/DATA/03_France_ByShape/A4/00114.jpg")
image_2 = imread("D:/svruchtel/DATA/03_France_ByShape/Ticket/00017.jpg")

In [8]:
print(image_1.shape)
print(image_2.shape)

(1024, 576, 4)
(768, 1024, 4)


In [25]:
# to solve blue pictures problem
image_1 = image_1[:, :, :3]
image_2 = image_2[:, :, :3]

In [26]:
print(image_1.shape)
print(image_2.shape)

(1024, 576, 3)
(768, 1024, 3)


In [27]:
image = image_1

In [28]:
larger_side = max(image.shape[:2])
smaller_side = min(image.shape[:2])
print(larger_side)
print(smaller_side)

1024
576


In [29]:
square_size = 128

In [30]:
proportion = larger_side / (square_size * 2)
print(proportion)

4.0


In [31]:
if (smaller_side / proportion) < square_size:
    proportion = smaller_side / (square_size * 2)

In [32]:
image = resize(image, (larger_side / proportion, smaller_side / proportion, 3))

C:\Program Files\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [33]:
image.shape

(256, 144, 3)

In [74]:
left_top_corner = image[:square_size, :square_size, :]
right_top_corner = image[:square_size, -square_size:, :]
left_bottom_corner = image[-square_size:, :square_size, :]
right_bottom_corner = image[-square_size:, -square_size:, :]

center_row = image.shape[0] / 2
center_col = image.shape[1] / 2
center_img = image[int(center_row - square_size / 2):int(center_row + square_size / 2),
                     int(center_col - square_size / 2):int(center_col + square_size / 2), :]


In [79]:
print(center_img.shape)

(128, 128, 3)


In [81]:
left_top_file_path = "D:/svruchtel/tmp/left_top/00114.jpg"
imsave(left_top_file_path, left_top_corner)

right_top_file_path = "D:/svruchtel/tmp/right_top/00114.jpg"
imsave(right_top_file_path, right_top_corner)

left_bottom_file_path = "D:/svruchtel/tmp/left_bottom/00114.jpg"
imsave(left_bottom_file_path, left_bottom_corner)

right_bottom_file_path = "D:/svruchtel/tmp/right_bottom/00114.jpg"
imsave(right_bottom_file_path, right_bottom_corner)

center_img_file_path = "D:/svruchtel/tmp/center/00114.jpg"
imsave(center_img_file_path, center_img)


C:\Program Files\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [100]:
# let's try to load and divide dataset

def divide_one_image_to_5(image, square_size):
    """
    image should be in .jpeg or .jpg extension
    """
    # if image is gray - convert it to rgb
    if len(image.shape) < 3:
        image = skimage.color.gray2rgb(image)
        
    # to solve blue pictures problem
    image = image[:, :, :3]
    
    larger_side = max(image.shape[:2])
    smaller_side = min(image.shape[:2])

    proportion = larger_side / (square_size * 2)
    if (smaller_side / proportion) < square_size:
        proportion = smaller_side / (square_size * 2)
    
    image = resize(image, (int(larger_side / proportion), int(smaller_side / proportion), 3))
    
    left_top_corner = image[:square_size, :square_size, :]
    right_top_corner = image[:square_size, -square_size:, :]
    left_bottom_corner = image[-square_size:, :square_size, :]
    right_bottom_corner = image[-square_size:, -square_size:, :]

    center_row = image.shape[0] / 2
    center_col = image.shape[1] / 2
    center_img = image[int(center_row - square_size / 2):int(center_row + square_size / 2),
                     int(center_col - square_size / 2):int(center_col + square_size / 2), :]
    
    return left_top_corner, right_top_corner, left_bottom_corner, right_bottom_corner, center_img
        
    
def make_5_images_for_all_dataset(source_directory, target_directory, square_size):
    for root, dirs, files in os.walk(source_directory):
        for name in files:
            
            file_path = os.path.join(root, name)
            
            image = imread(file_path)
    
            left_top, right_top, left_bottom, right_bottom, center_img = divide_one_image_to_5(image, square_size)
        
            _, category_folder_name = os.path.split(root)
            
            category_folder_path = os.path.join(target_directory, category_folder_name)
            # create this folder in target_directory
            if not os.path.exists(category_folder_path):
                    try:
                        os.makedirs(category_folder_path)
                    except:
                        print("CANNOT CREATE DIRECTORY")
                        break
                        
            # than create 5 more folders in this new folder
            new_dirs = ["left_top", "right_top", "left_bottom", "right_bottom", "center"]
            
            for new_dir in new_dirs:
                new_dir_path = os.path.join(category_folder_path, new_dir)
                if not os.path.exists(new_dir_path):
                    try:
                        os.makedirs(new_dir_path)
                    except:
                        print("CANNOT CREATE DIRECTORY")
                        break
                        
            lt_path = os.path.join(os.path.join(category_folder_path, new_dirs[0]), name)
            imsave(lt_path, left_top)
            rt_path = os.path.join(os.path.join(category_folder_path, new_dirs[1]), name)
            imsave(rt_path, right_top)
            lb_path = os.path.join(os.path.join(category_folder_path, new_dirs[2]), name)
            imsave(lb_path, left_bottom)
            rb_path = os.path.join(os.path.join(category_folder_path, new_dirs[3]), name)
            imsave(rb_path, right_bottom)
            ct_path = os.path.join(os.path.join(category_folder_path, new_dirs[4]), name)
            imsave(ct_path, center_img)
            

In [101]:
make_5_images_for_all_dataset("D:/svruchtel/make_5_source", "D:/svruchtel/make_5_target", 128)

C:\Program Files\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [102]:
%%time

# the function seems to be ready, let's try to work with it
make_5_images_for_all_dataset(source_directory="D:/svruchtel/DATA/France_New_ByShape",
                             target_directory="D:/svruchtel/PREPROCESSED_DATA/France_New_ByShape_128_3dim",
                             square_size=128)


C:\Program Files\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/PREPROCESSED_DATA/France_New_ByShape_128_3dim\Ticket\left_top\003699.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/PREPROCESSED_DATA/France_New_ByShape_128_3dim\Ticket\left_top\003716.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/PREPROCESSE

Wall time: 28min 41s


In [103]:
img_test = imread("D:/svruchtel/PREPROCESSED_DATA/France_New_ByShape_128_3dim/Ticket/center/000455.jpg")
print(img_test.shape)

(128, 128, 3)


In [104]:
%%time

# the function seems to be ready, let's try to work with it
make_5_images_for_all_dataset(source_directory="D:/svruchtel/DATA/France_New_ByShape",
                             target_directory="D:/svruchtel/PREPROCESSED_DATA/France_New_ByShape_224_3dim",
                             square_size=224)

C:\Program Files\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/PREPROCESSED_DATA/France_New_ByShape_224_3dim\Ticket\left_top\003716.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/PREPROCESSED_DATA/France_New_ByShape_224_3dim\Ticket\right_top\003716.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/PREPROCESS

Wall time: 41min 58s


In [105]:
img_test = imread("D:/svruchtel/PREPROCESSED_DATA/France_New_ByShape_224_3dim/Ticket/center/000455.jpg")
print(img_test.shape)

(224, 224, 3)


In [106]:
%%time

# the function seems to be ready, let's try to work with it
make_5_images_for_all_dataset(source_directory="D:/svruchtel/DATA/France_New_ByMeaning",
                             target_directory="D:/svruchtel/PREPROCESSED_DATA/France_New_ByMeaning_128_3dim",
                             square_size=128)

C:\Program Files\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/PREPROCESSED_DATA/France_New_ByMeaning_128_3dim\Transport\left_top\003699.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/PREPROCESSED_DATA/France_New_ByMeaning_128_3dim\Transport\left_top\003716.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/P

Wall time: 26min 51s


In [107]:
%%time

# the function seems to be ready, let's try to work with it
make_5_images_for_all_dataset(source_directory="D:/svruchtel/DATA/France_New_ByMeaning",
                             target_directory="D:/svruchtel/PREPROCESSED_DATA/France_New_ByMeaning_224_3dim",
                             square_size=224)

C:\Program Files\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/PREPROCESSED_DATA/France_New_ByMeaning_224_3dim\Transport\left_top\003716.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/PREPROCESSED_DATA/France_New_ByMeaning_224_3dim\Transport\right_top\003716.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/

Wall time: 37min 58s


In [108]:
%%time

# the function seems to be ready, let's try to work with it
make_5_images_for_all_dataset(source_directory="D:/svruchtel/DATA/France_New_ByMeaningLarge",
                             target_directory="D:/svruchtel/PREPROCESSED_DATA/France_New_ByMeaningLarge_128_3dim",
                             square_size=128)

C:\Program Files\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/PREPROCESSED_DATA/France_New_ByMeaningLarge_128_3dim\AllTransport\left_top\003699.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/PREPROCESSED_DATA/France_New_ByMeaningLarge_128_3dim\AllTransport\left_top\003716.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning

Wall time: 28min 4s


In [109]:
%%time

# the function seems to be ready, let's try to work with it
make_5_images_for_all_dataset(source_directory="D:/svruchtel/DATA/France_New_ByMeaningLarge",
                             target_directory="D:/svruchtel/PREPROCESSED_DATA/France_New_ByMeaningLarge_224_3dim",
                             square_size=224)

C:\Program Files\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/PREPROCESSED_DATA/France_New_ByMeaningLarge_224_3dim\AllTransport\left_top\003716.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarning: D:/svruchtel/PREPROCESSED_DATA/France_New_ByMeaningLarge_224_3dim\AllTransport\right_top\003716.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Program Files\Anaconda3\lib\site-packages\skimage\io\_io.py:129: UserWarnin

Wall time: 36min 46s
